In [1]:

# Загрузить необходимые пакеты для работы
!pip install geopandas
!pip install pandas 
!pip install jupyter_bokeh


In [2]:
# Импорт библиотеки для работы
import geopandas as gpd 
import pandas as pd 
import json 
import glob 
import os 
import numpy as np
import requests
from ipywidgets import interact 
from bokeh.resources import INLINE 
from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure 
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar 
from bokeh.palettes import brewer,mpl
from bokeh.io import curdoc, output_notebook 
from bokeh.models import Slider, HoverTool,CustomJS 
from bokeh.layouts import widgetbox, row, column
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
#output_notebook()

In [3]:
# Создание класса карты со следующими свойствами:
# Заголовок -title
# Патлетт -palette
# Линейный цветовой картограф -color_mapper
# всплывающие подсказки -hover
# фигура -fig
class Map:
    # Метод конструктора
    def __init__(self):
        self.title="To do some thing title for map"
        self.palette=mpl['Plasma'][10]
        self.color_mapper=LinearColorMapper(palette = self.palette, low = 0, high = 10)
        self.hover=HoverTool(tooltips = [ ('Country','@country'),('Score', '@Score')])
        self.color_bar=ColorBar(color_mapper=self.color_mapper, label_standoff=8,width = 500, height = 20, border_line_color=None,location = (0,0), orientation = 'horizontal')
        self.fig=figure(title = self.title, plot_height = 600 , plot_width = 950, toolbar_location = None, tools = [self.hover])
    # Метод  установки значения для патлетта
    def setPalette(self,palette):
        self.palette=palette
    # Метод  установки значения для заголовок
    def setTitle(self,title):
        self.title=title
    # Метод  установки значения для линейного цветовгого картографа
    def setColorMapper(self, palette, low, high):
         self.color_mapper=LinearColorMapper(palette = self.palette, low = low, high = high)
    # Метод  установки значения для всплывающих подсказок
    def setHover(self,label, value):
        self.hover=HoverTool(tooltips=(label, value))
    # Метод  установки значения для цветной полосы
    def setColorBar(self,color_mapper,label_standoff, w, h, color, point, orientation):
        self.color_bar=ColorBar(color_mapper=color_mapper, label_standoff=label_standoff,width = w, height = h, border_line_color=color,location = point, orientation = orientation)
    # Метод  установки значения для фигуры
    def setFigure(self,title,ph,pw,toolbar,hover):
        self.fig=figure(title = title, plot_height = ph , plot_width = pw, toolbar_location = None, tools = [hover])

# Создание класса worldHappines наследования свойства с класса Map
class worldHappines(Map):
    # Предобработка данных
    def processingData(self):
        shapefile = '110m_cultural/ne_110m_admin_0_countries.shp'
        gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
        gdf.columns = ['country', 'country_code', 'geometry']
        listData_1 = ['2019.csv', '2018.csv']
        listData_2 = ['2015.csv', '2016.csv']
        df1 = pd.concat((pd.read_csv(f).assign(Year=os.path.basename(f[0:4])) for f in listData_1), sort=False)
        df2 = pd.concat((pd.read_csv(f).assign(Year=os.path.basename(f[0:4])) for f in listData_2), sort=False)
        df3 = pd.read_csv('2017.csv').assign(Year="2017")
        df1['Country'] = df1['Country or region']
        df2['Score'] = df2['Happiness Score']
        df3['Score'] = df3['Happiness.Score']
        countries = pd.concat([df1[['Country', 'Score', 'Year']], df2[['Country', 'Score', 'Year']], df3[['Country', 'Score', 'Year']]], sort=False)
        countries = countries.replace('United States', 'United States of America')
        return gdf,countries
    # Метод конструктора без параметров
    def __init__(self):
        gdf,countries=self.processingData();
        self.gdf=gdf
        self.countries=countries
        # вызывает конструктора класса отца
        Map.__init__(self)
    # Получение данные в формате 
    def json_data(self,selectedYear,gdf,countries):
        year = selectedYear
        df_yr = countries[countries['Year'] == year]
        merged = gdf.merge(df_yr, left_on ='country', right_on ='Country', how ='left')
        merged['Score'] = merged.Score.fillna("N/A")
        merged_json = json.loads(merged.to_json())
        json_data = json.dumps(merged_json)
        return json_data
    # Полимозфизма метода setFigure из отца класса
    def setFigure(self,xgrid,ygrid,fontText,src,fillColor,lineColor,lineWitdh,alpha,colorbar,valueColorbar):
        self.fig.xgrid.grid_line_color = xgrid
        self.fig.ygrid.grid_line_color = ygrid
        self.fig.title.text_font_size = fontText
        self.fig.patches('xs','ys', source = src,fill_color = fillColor, line_color = lineColor, line_width = lineWitdh, fill_alpha = alpha)
        self.fig.add_layout(colorbar, valueColorbar)
    # Рисование карты по году
    def draw_map(self,year):
        self.title='Happiness Score by Country  '+ str(year)
        self.jsondata=self.json_data(str(year),self.gdf,self.countries)
        self.geosource = GeoJSONDataSource(geojson = self.jsondata)
        super().setFigure(self.title,600,950,None,self.hover)
        self.setFigure(None,None,'12pt',self.geosource,{'field' :'Score', 'transform' : self.color_mapper},'white',0.5,1,self.color_bar,'below')
        layout = column(self.fig,widgetbox())
        curdoc().add_root(layout)
        output_notebook(INLINE)
        show(layout,notebook_handle=True)

In [4]:
# Вызывает класс worldHappines
wh=worldHappines()
# Интеракт для выбрана года, отображающего уровня счастья стран в исходном. 
interact(wh.draw_map,year=['2015','2016','2017','2018','2019'])

C:\Users\Tran Van Thai\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\array.py:85: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


interactive(children=(Dropdown(description='year', options=('2015', '2016', '2017', '2018', '2019'), value='20…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>